In [69]:
import pandas as pd
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
import torch
import os
from datasets import load_dataset 
from nlpaug.util import Action

from langchain.agents import create_pandas_dataframe_agent, load_tools
from langchain.llms import OpenAI
from getpass import getpass

In [42]:
text = "I think that all Ethiopians should deserve jail time."

In [43]:
print(text)

I think that all Ethiopians should deserve jail time.


In [44]:
augmentor = naw.AntonymAug()
augmentor.augment(text.split(" ")[4])

['Ethiopians']

Both Synonym and Antonym augmentation do not work very well with race. Use embedding context

In [45]:
aug = naw.ContextualWordEmbsAug(
    model_path='roberta-base', action="substitute")
aug.augment(text.split(" ")[4])

['Ethiopians']

Not very successful; need to do it manually

In [62]:
os.environ['OPENAI_API_KEY'] = getpass("OpenAI Key:")

OpenAI Key: ········


In [65]:
agent = OpenAI(temperature=0)

In [ ]:
agent("generate 50 ethnicities and races")

In [67]:
bias_elements = [
    "African",
    "Afro-Caribbean",
    "Afro-Latino",
    "Ainu",
    "Amazigh",
    "Ashkenazi Jewish",
    "Australian Aboriginal",
    "Basque",
    "Bantu",
    "Bengali",
    "Berber",
    "Black",
    "Chinese",
    "Cree",
    "Cuban",
    "Druze",
    "Dutch",
    "English",
    "Filipino",
    "Finnish",
    "French",
    "Gaelic",
    "German",
    "Greek",
    "Guatemalan",
    "Han Chinese",
    "Hausa",
    "Hawaiian",
    "Hispanic/Latino",
    "Hmong",
    "Hopi",
    "Inuit",
    "Italian",
    "Japanese",
    "Jewish",
    "Karen",
    "Khmer",
    "Korean",
    "Maori",
    "Maya",
    "Mongolian",
    "Native American",
    "Nigerian",
    "Nubian",
    "Pashtun",
    "Persian",
    "Quechua",
    "Romani",
    "Sami",
    "Somali",
    "Tajik",
    "Tamil",
    "Tatar",
    "Thai",
    "Tibetan",
    "Tuareg",
    "Turkish",
    "Uighur",
    "Ukrainian",
    "Vietnamese",
    "Yakut",
    "Yoruba",
    "Zulu",
    "Albanian",
    "Arab",
    "Armenian",
    "Assyrian",
    "Aymara",
    "Balinese",
    "Bashkir",
    "Belizean",
    "Bolivian",
    "Bosniak",
    "Bulgarian",
    "Cambodian",
    "Cameroonian",
    "Catalan",
    "Chamorro",
    "Chechen",
    "Cherokee",
    "Chuvash",
    "Coptic",
    "Corsican",
    "Crimean Tatar",
    "Croatian",
    "Czech",
    "Danish",
    "Dinka",
    "Ecuadorian",
    "Estonian",
    "Ethiopian",
    "Fijian",
    "Georgian",
    "Gujarati",
    "Haitian",
    "Hazaras",
    "Ibo",
    "Icelandic",
    "Indigenous Australian",
    "Indigenous Malaysian",
    "Iraqi",
    "Iroquois",
    "Kurdish",
    "Latvian",
    "Lebanese",
    "Lithuanian",
    "Macedonian",
    "Malay",
    "Maldivian",
    "Maltese",
    "Maasai",
    "Mende",
    "Mien",
    "Mizrahi Jewish",
    "Monguor",
    "Moroccan",
    "Navajo",
    "Nenets",
    "Nepali",
    "Norwegian",
    "Pakistani",
    "Palestinian",
    "Papua New Guinean",
    "Parsi",
    "Peruvian",
    "Polish",
    "Portuguese",
    "Punjabi",
    "Roma",
    "Samoan",
    "Scots",
    "Sindhi",
    "Slovak",
    "Slovene",
    "Sorbian",
    "Sudanese",
    "Swedish",
]

In [71]:
base_dataset = load_dataset("ccdv/govreport-summarization")
base_dataset

DatasetDict({
    train: Dataset({
        features: ['report', 'summary'],
        num_rows: 17517
    })
    validation: Dataset({
        features: ['report', 'summary'],
        num_rows: 973
    })
    test: Dataset({
        features: ['report', 'summary'],
        num_rows: 973
    })
})

Procedure:
- Iterate through each data entry, and do spaCy named entity recognition for race.
- Once found, take entry, and copy in word from list of bias elements to generate new entry.
- Each entry will then be augmented in size 